In [7]:
import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET
from pathlib import Path

from sklearn.model_selection import train_test_split
from pandas import read_csv
from PIL import Image

In [8]:
IMG_DIR = 'images-anot/'
CROPPED_DIR = 'cropped/'

In [9]:
class KerasnetUtils:
    @staticmethod
    def xml_to_df(path):
        """Iterates through all .xml files (generated by labelImg) in a given directory and combines them in a single Pandas dataframe.

        Parameters:
        ----------
        path : {str}
            The path containing the .xml files
        Returns
        -------
        Pandas DataFrame
            The produced dataframe
        """

        xml_list = []
        for xml_file in glob.glob(path + '/*.xml'):
            tree = ET.parse(xml_file)
            root = tree.getroot()
            for member in root.findall('object'):
                value = (root.find('filename').text,
                        int(member[4][0].text),
                        int(member[4][1].text),
                        int(member[4][2].text),
                        int(member[4][3].text),
                        member[0].text
                )
                xml_list.append(value)

        column_name = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class']
        xml_df = pd.DataFrame(xml_list, columns=column_name)
        return xml_df

    """
    Crops annotated areas into separate picture files.

    csv : annotation file to use
    dest: save results in this directory
    """
    @staticmethod
    def crop_anot(dest: str, csv: str):
        # load into dataframe
        df = read_csv(csv)
        
        # get cropped area from each pic and save
        for i,row in df.iterrows():
            filename = f"{IMG_DIR}{row['filename']}"
            img = Image.open(filename)
            area = (row['xmin'], row['ymin'], row['xmax'], row['ymax'])
            cropped_img = img.crop(area)
            
            # make directory for each class
            SAVE_DIR =  f"{CROPPED_DIR}{row['class']}/"
            Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
            cropped_img.save(f"{SAVE_DIR}{i}.png", 'PNG')

In [10]:
utils = KerasnetUtils()
df = utils.xml_to_df(IMG_DIR)
df.to_csv('labels.csv', index=None)

utils.crop_anot(CROPPED_DIR, 'labels.csv')

# Training Inception v3 classifier

In [11]:
!pip install tensorboard tensorflow_hub
!pip install matplotlib

In [12]:
import itertools
import os
from datetime import datetime

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

/home/leix/.pyenv/versions/3.7.6/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/leix/.pyenv/versions/3.7.6/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/leix/.pyenv/versions/3.7.6/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/leix/.pyenv/versions/3.7.6/lib/python3.7/site-

TF version: 2.0.0-beta1
Hub version: 0.8.0
GPU is NOT AVAILABLE


In [13]:
handle_base, pixels = ("inception_v3", 299)
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

BATCH_SIZE = 64
EPOCHS = 50

Using https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4 with input size (299, 299)


## Training directory:

In [14]:
data_dir = 'cropped/'

## Image preprocessing:

In [15]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

do_data_augmentation = True

In [16]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)


if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True,
        width_shift_range=0.2, height_shift_range=0.2,
        shear_range=0.2, zoom_range=0.2,
        **datagen_kwargs)
else:
    train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

Found 101 images belonging to 6 classes.
Found 412 images belonging to 6 classes.


## Hyperparameters:

In [24]:
retrain_inception_v3 = False
L2_penalty = 0.0001
dropout = 0.2
lrate = 0.005
momentum = 0.9

In [25]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=retrain_inception_v3),
    tf.keras.layers.Dropout(rate=dropout),
    tf.keras.layers.Dense(train_generator.num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(L2_penalty)
                         )
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 2048)              21802784  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12294     
Total params: 21,815,078
Trainable params: 12,294
Non-trainable params: 21,802,784
_________________________________________________________________


In [19]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=lrate, momentum=momentum), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

#### Save models during training:

In [20]:
checkpoint_filepath ="checkpoints/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=False,
    mode='max'
)

### Hook up tensorboard:

In [21]:
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

#### Put together callbacks:

In [23]:
callback_list = [checkpoint, tensorboard_callback]

#### Start training:

In [ ]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
hist = model.fit(
    train_generator,
    epochs=EPOCHS, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    callbacks=callback_list,
    validation_steps=validation_steps).history

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

In [ ]:
saved_model_path = "models/color_classifier"
tf.saved_model.save(model, saved_model_path)